1. Проверить как идет обучение при различных параметрах модели для выборки "Ирисы Фишера":
    
    - Измените размер batch_size : 1, 10, 20, 50
    
    - измените значение lyambd: 0.01, 0.1, 0.5, 1 
    
    - измените число нейронов в скрытом слое : 1, 5, 50

2. Сделайте несколько перезапусков каждой модели (10 перезапусков с вычислением среднего и дисперсии для оценки точности).

3. Оцените изменения параметров статистик точности при изменении параметров модели. 




In [0]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [0]:
def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]] = 1.
    return binarized

def from_one_hot(Y):
    arr = np.zeros((len(Y), 1))

    for i in range(len(Y)):
        l = layer2[i]
        for j in range(len(l)):
            if(l[j] == 1):
                arr[i] = j+1
    return arr


def sum_neuron(x=None, w=None):
    return np.dot(w.T,x.T )

def sigmoid_complex_neuron(x=None, w=None, bias=0, lambda_=1):
    s = sum_neuron(x=x, w=w).T
    y = 1 / (1 + np.exp(-s / lambda_))
    return y

def sigmoid_deriv(g):
    return g * (1 - g)

def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

In [0]:
def custom_net(X_train, X_test, y_train, y_test, epochs=5000, neurons=5, lambda_=0.1,  batch_size=10, n=0.01):

    w0 = 2*np.random.random((4, neurons)) - 1
    w1 = 2*np.random.random((neurons, 3)) - 1

    errors = []
    errors_test = []

    
    
    layer0 = X_train
    layer1 = sigmoid_complex_neuron(x=layer0, w=w0)
    layer2 = sigmoid_complex_neuron(x=layer1, w=w1)
    layer2_error = (y_train - layer2)

 
    Q = np.mean(np.abs(layer2_error)).copy()
    
    for i in range(epochs):

        j = np.random.randint(0, X_train.shape[0], (batch_size))
        layer0 = X_train[j,:]
        layer1 = sigmoid_complex_neuron(x=layer0, w=w0)
        layer2 = sigmoid_complex_neuron(x=layer1, w=w1)

        layer2_error = y_train[j] - layer2

        
        
        Q = Q * (1 - lambda_) + lambda_ * np.mean(np.abs(layer2_error))
        errors.append(Q.copy())

        dlayer2 = sigmoid_deriv(layer2)
        dlayer1 = sigmoid_deriv(layer1)
 
        layer2_delta = layer2_error*dlayer2 
    
    
        layer1_error = np.dot(layer2_delta, w1.T)
                
        layer1_delta = layer1_error * dlayer1 
                 
        dw1 = np.dot(layer2_delta.reshape((layer2_delta.shape[1], layer2_delta.shape[0])), layer1) * n
        dw0 = np.dot(layer1_delta.reshape((neurons, layer1_delta.shape[0])), layer0) * n
       
        w1 += dw1.T
        w0 += dw0.T
        
        error = np.mean(np.abs(layer2_error))
        accuracy = (1 - error.copy()) * 100

        layer0_test = X_test
        layer1_test = sigmoid_complex_neuron(x=layer0_test, w=w0)
        layer2_test = sigmoid_complex_neuron(x=layer1_test, w=w1)
        
        
        layer2_error_test = y_test - layer2_test
    
        error = np.mean(np.abs(layer2_error_test))
        errors_test.append(error.copy())
    
        accuracy_test = (1 - error.copy()) * 100
        
    return accuracy, accuracy_test, w0, w1, errors_test, errors
    


In [63]:
iris_ = load_iris()
iris_data = iris_.data
iris_target = iris_.target
columns = iris_.feature_names
df = pd.DataFrame(data=iris_data, columns=columns)
df['target'] = iris_.target

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [64]:
X = normalize(df[columns].values)
y = df['target'].values

y = y.flatten()
y = to_one_hot(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

print(X_train.shape)

(100, 4)


In [0]:
batch_sizes = [1, 10, 20, 50]
lambdas = [0.01, 0.1, 0.5, 1]
n_neurons = [1, 5, 50]
n_models = 10

In [66]:
results = {}
for size in batch_sizes:
  result = []
  for i in range(n_models):
    accuracy = custom_net(X_train, X_test, y_train, y_test, batch_size=size)[1]
    result.append(accuracy)
  results[size] = {'mean': np.mean(result), 'variance': np.var(result, ddof=1)}

results

{1: {'mean': 55.3799202370801, 'variance': 0.15752141613178056},
 10: {'mean': 55.503998140194525, 'variance': 0.12091488778198108},
 20: {'mean': 55.32386941526954, 'variance': 0.12858001777167855},
 50: {'mean': 55.440482678044624, 'variance': 0.07654968420449268}}

In [67]:
results = {}
for l in lambdas:
  result = []
  for i in range(n_models):
    accuracy = custom_net(X_train, X_test, y_train, y_test, lambda_=l)[1]
    result.append(accuracy)
  results[l] = {'mean': np.mean(result), 'variance': np.var(result, ddof=1)}

results

{0.01: {'mean': 55.321146755131664, 'variance': 0.05454993940983452},
 0.1: {'mean': 55.48199494289427, 'variance': 0.19241513038765326},
 0.5: {'mean': 55.42472397719814, 'variance': 0.06414892866753659},
 1: {'mean': 55.42156788286017, 'variance': 0.10308838762381656}}

In [68]:
results = {}
for n in n_neurons:
  result = []
  for i in range(n_models):
    accuracy = custom_net(X_train, X_test, y_train, y_test, neurons=n)[1]
    result.append(accuracy)
  results[n] = {'mean': np.mean(result), 'variance': np.var(result, ddof=1)}

results

{1: {'mean': 55.381600152164154, 'variance': 0.10769437505098248},
 5: {'mean': 55.23952513321363, 'variance': 0.07010429928462668},
 50: {'mean': 56.57086149001515, 'variance': 1.325467481793543}}

По результатам видим, что от параметра `batch_size` точность практически не зависит. В случае с параметром `lambda_` точность связана больше, т.к. дисперсия меньше, а значит разница в меньшей степени объясняется случайным распределением. Количество нейронов дает прирост в точности, но при этом при большом количестве нейронов увеличивается разброс в результатах, что, кажется, ожидаемо.